In [1]:
%pwd

'c:\\Users\\vamsh\\Desktop\\Vamshi PC\\Machine Learning and DSA\\Self Projects\\medic.ai\\research'

In [2]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    docs = loader.load()
    return docs

In [4]:
extracted_data = load_pdf(data='..//Data/')


In [5]:
# extracted_data

# Chunking : 
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"length of chunks : {len(text_chunks)}")

length of chunks : 7152


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings 

def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hf_embeddings()
print(embeddings)

C:\Users\vamsh\AppData\Local\Temp\ipykernel_10792\1856694351.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vamsh\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [7]:
test = embeddings.embed_query("hey there")
print(f"Length : {len(test)}") # 384 for 384 dimensions

Length : 384


In [8]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)


In [11]:

index_name = "medicoai"

pc.create_index(
    name=index_name,
    dimension= 384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '619dcdb330b94053ee10005eac7064cb', 'Date': 'Sat, 19 Oct 2024 10:54:45 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [9]:
from langchain_pinecone import PineconeVectorStore

In [13]:

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding= embeddings
)


KeyboardInterrupt: 

In [18]:
# Above cell will store embeddings of every single chunk of our document.

# Done and dusted, 7440 chunks in pinecone.

In [14]:
# Getting vector embeddings from pinecone index 
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
docsearch

In [15]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [16]:
retrieved_docs = retriever.invoke("What is cardiac arrest ?")

In [17]:
retrieved_docs

[Document(metadata={'page': 916.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='brain is being deprived of oxygenated blood, a medicalemergency exists.\nDiagnosis\nDiagnosis of sudden cardiac death is made when\nthere is a sudden loss of consciousness, breathing stops,and there is no effective heart beat.\nTreatment\nWhen sudden cardiac death occurs, the first priority\nis to establish the flow of oxygenated blood to the brain.The next priority is to restore normal rhythm to the heart.Forcing air into the mouth will get oxygen into the lungs.Compressing the chest simulates a pumping heart andwill get some blood flow to the lungs, brain, and coro-nary arteries. This method is called cardiopulmonary\nresuscitation (CPR ). When trained help arrives, they'),
 Document(metadata={'page': 916.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='due to heart problems, which occurs within one hourfrom the start of any ca

In [18]:
from llama_index.llms.groq import Groq 

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

llm = Groq(
    api_key= GROQ_API_KEY,
    model = "llama3-70b-8192",
    temperature=0.3,
    max_retries=3
)

resp = llm.complete("What is the capital of France?")
print(resp)

The capital of France is Paris.


In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [35]:
system_prompt = (
    "You are a medical AI assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "If you don't know the answer, say that you don't know, don't assume answers"
    "Keep your answers in three lines (short and concise)"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)


In [33]:
# !pip install -U langchain
!pip install --upgrade langchain-community langchain-experimental langchain-openai langchain-pinecone

  Using cached langchain_experimental-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   --------------------- ------------------ 1.3/2.4 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.51.0
    Uninstalling openai-1.51.0:
      Successfully uninstalled openai-1.51.0
  Attempting uninstall: langchain-pinecone
    Found existing installation: langchain-pinecone 0.1.3
    Uninstalling langchain-pinecone-0.1.3:
      Successfully uninstalled langchain-pinecone-0.1.3
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.25
    Uninstalling langchain-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.2.7 requires llama-index-core<0.11.0,>=0.10.41, but you have llama-index-core 0.11.15 which is incompatible.
llama-index-agent-openai 0.2.7 requires llama-index-llms-openai<0.2.0,>=0.1.5, but you have llama-index-llms-openai 0.2.10 which is incompatible.
llama-index-multi-modal-llms-openai 0.1.6 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.15 which is incompatible.
llama-index-multi-modal-llms-openai 0.1.6 requires llama-index-llms-openai<0.2.0,>=0.1.1, but you have llama-index-llms-openai 0.2.10 which is incompatible.
llama-index-program-openai 0.1.6 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.15 which is incompatible.
llama-index-program-openai 0.1.6 requires llama-index-llms-openai<0.2.0,>=0.1.1, but you have llama-ind

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
print(type(question_answer_chain))
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
print(type(rag_chain))

# response = rag_chain.invoke({"question": "What is Acne?"})
# print(response["answer"]) 

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'llama_index.llms.groq.base.Groq'>

# Ollama Llama 3.1 8b

In [36]:
from langchain_community.chat_models import ChatOllama 
client = ChatOllama(
    # base_url = "http://localhost:11434/v1",
    # api_key = "ollama", # required, but unused
    model = "llama3.1:8b"
)

prompt = ChatPromptTemplate.from_template(
    "you are an AI Medical Expert for question and answering tasks"
    "You are given a question and a context. You need to answer the question based on the context."
    "If the context doesn't contain the answer, you need to say 'I don't know'"
    "Context: {context}"
    "Question: {question}"
    "Answer: "
)

C:\Users\vamsh\AppData\Local\Temp\ipykernel_10792\1406640509.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the langchain-ollama package and should be used instead. To use it run `pip install -U langchain-ollama` and import as `from langchain_ollama import ChatOllama`.
  client = ChatOllama(


In [45]:
retreiver = docsearch.as_retriever(
    search_type="similarity", search_kwargs={"k": 3}
)

In [68]:
from pydantic import BaseModel

# Define your model and set arbitrary_types_allowed to True
class ConfigWithArbitraryTypes(BaseModel):
    class Config:
        arbitrary_types_allowed = True

# Now create the chain again
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Build the chain with arbitrary types allowed
chain = ({"context": retriever, "question": RunnablePassthrough()} 
         | prompt 
         | client 
         | StrOutputParser())


In [69]:
chain.invoke("What is Acne?")

RuntimeError: no validator found for <class 'langchain_core.runnables.base.Runnable'>, see `arbitrary_types_allowed` in Config

In [77]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings  # You can use other embeddings
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
from langchain.schema import HumanMessage

# ollama_llm = Ollama(model="llama3.1:8b")

template = """
You are a highly experienced medical expert.

Based on the patient's question, please provide a detailed and accurate response.
Include possible side effects, dosage information, and any warnings or contraindications where applicable.

Question: {question}
Context: {context}
Answer: 
"""

prompt = ChatPromptTemplate.from_template(template)

def ask_question_with_retrieval(question):
    docs = retreiver.get_relevant_documents(question)
    context = " ".join([doc.page_content for doc in docs])
    prompt_input = prompt.format(context=context, question=question)
    messages = [HumanMessage(content=prompt_input)]
    response = client.generate(messages)
    return response

question = "What are the side effects of paracetamol?"
response = ask_question_with_retrieval(question)
print(response)


ValueError: Received unsupported message type for Ollama.